In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

## IMDB 데이터셋 다운로드

In [ ]:
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

매개변수 num_words=10000은 훈련 데이터에서 가장 많이 등장하는 상위 10000개의 단어를 선택한다.

## 데이터 탐색

이 데이터셋의 샘플은 전처리된 정수 배열이다. 이 정수는 영화 리뷰에 나오는 단어를 나타낸다. 레이블은 정수 0 또는 1이다. 0은 부정적인 리뷰 1은 긍정적인 리뷰

In [ ]:
print("훈련 샘플: {}, 레이블: {}".format(len(train_data), len(train_labels)))

In [ ]:
print(train_data[0])

In [ ]:
len(train_data[0]), len(train_data[1])

## 정수를 단어로 다시 변환

정수와 문자열을 매핑하는 딕셔너리 객체에 질의하는 helper 함수를 만든다. 

In [ ]:
# 단어와 정수 인덱스를 매핑한 딕셔너리
word_index = imdb.get_word_index()

# 처음 몇 개 인덱스는 사전에 정의되어 있습니다
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [ ]:
decode_review(train_data[0])

## 데이터 준비

리뷰 -정수배열-는 신경망에 주입하기 전에 텐서로 변환되어야 한다.

턴서 변환 방법

> 원-핫 인코딩은 정수 배열을 0과 1로 이루어진 벡터로 변환한다. 그 다음 실수 벡터 데이터를 다룰수 있는 Dense층을 신경망의 첫번째 층으로 사용한다. 이 방법은 num_words * num_reviews 크기의 행렬이 필요하기 때문에 메모리를 많이 사용한다.

> 정수 배열의 길이가 모두 같도록 패딩을 추가해서 max_length * num_reviews 크기의 정수 텐서를 만든다. 이 형태의 텐서를 다룰 수 있는 임베딩 층을 첫번째 층으로 사용할 수 있다.

여기선 2번째 방법을 사용한다.

영화 리뷰의 길이가 같아야하므로 pad_sequences함수를 사용한다.

In [ ]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

In [ ]:
len(train_data[0]), len(train_data[1])

In [ ]:
# 패딩된 첫 리뷰 내용
print(train_data[0])

## 모델 구성
신경망은 layer를 쌓아서 만든다. 이 때 두가지를 결정해야 한다.

> 모델에서 얼마나 많은 *층*을 사용할 것인가?

> 각 층에서 얼마나 많은 *은닉 유닛*을 사용할 것인가?

In [ ]:
# 입력 크기는 영화 리뷰 데이터셋에 적용된 어휘 사전의 크기입니다(10,000개의 단어)
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16, input_shape=(None,)))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

1. 첫번째 층은 embeding 층이다. 이 층은 정수로 인코딩된 단어를 입력받고 각 단어 인덱스에 해당하는 임베딩 벡터를 찾는다. 이 벡터는 모델이 훈련되면서 학습된다. 이 벡터는 출력 배열에 새로운 차원으로 추가된다. 최종 차원은 (batchm sequence, embedding)
embedding은 자연어를 수치화 하기위해 필요한 layer

2. 그 다음 GlobalAveragePooling1D 층은 sequence 차원에 대해 평균을 계산하여 각 샘플에 대해 고정된 길이의 출력 벡터를 반환한다. 이는 길이가 다른 입력을 다루는 가장 간단한 방법이다.

3. 이 고정 길이의 출력 벡터는 16개의 은닉 유닛을 가진 Dense를 거친다.

4. 마지막 층은 하나의 출력 노드를 가진 Dense층이다. sigmoid 활성화 함수를 사용하여 0과 1사이의 실수를 출력한다. 이 값은 확률 또는 신뢰도를 나타낸다. \

## 은닉 유닛

위 모델에는 입력과 출력 사이에 두개의 "은닉"층이 있다. 출력의 개수는 층이 가진 표현 공간의 차원이 된다. 다른 말로 하면 내부 표현을 학습할 때 허용되는 네트워크 자유도의 양이다.

모델에 많은 은닉 유닛과 층이 있다면 네트워크는 더 복잡한 표현을 학습할 수 있다. 하지만 네트워크의 계산 비용이 ㅁ낳이 들고 원치 않는 패턴을 학습할 수도 있다. 이런 표현은 훈련 데이터의 성능을 향상시키지만 테스트 데이터에서는 그렇지 못한다. 이를 *과대적합*이라고 부른다. 

## 손실 함수와 옵티마이저

모델이 훈련하기 위해 필요한 것이 손실함수와 옵티마이저이다. 

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## 검증 세트 만들기

모델을 훈련할 때 모델이 만난 적 없는 데이터에서 정확도를 확인하는 것이 좋다. 원본 데이터에서 10,000개의 샘플을 떼어내어 검증세트를 만든다. 

In [ ]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

## 모델 훈련

이 보델을 312개의 샘플로 이루어진 미니 배치에서 40번의 에포크동안 훈련한다. x_train과 y_train 텐서에 있는 모든 샘플에 대해 40번 반복한다.

In [ ]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

## 모델 평가

모델의 성능은 손실과 정확도로 평가한다.

In [ ]:
results = model.evaluate(test_data,  test_labels, verbose=2)

print(results)

## 정확도와 손실 그래프

model.fit()은 history 객체를 반환한다. 여기에는 훈련하는 동안 일어나는 모든 정보가 dictionary에 들어가있다.

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
import matplotlib.pyplot as plt

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo"는 "파란색 점"입니다
plt.plot(epochs, loss, 'bo', label='Training loss')
# b는 "파란 실선"입니다
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.clf()   # 그림을 초기화합니다

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

점선은 훈련 손실과 훈련 정확도, 실선은 검증 손실과 검증 정확도이다.

훈련 손실은 에포크마다 감소하고 훈련 정확도는 증가한다. 경사 하강법 최적화를 사용할 때마다 볼 수 있는 현상이다. 매 반복마다 최적화 대상의 값을 최소화한다.

그러나 검증 손실과 검증 정확도에서는 그렇지 못하다. 약 20번쨰 에포크 이후 최적짐인 듯 하다. 이는 *과대적합* 때문이다. 이전에 본 적 없는 데이터보다 훈련 데이터에서 잘 작동한다. 이 지점부터 모델이 과도하게 최적화되어 테스트 데이터에서 일반화되기 어려운 훈련데이터의 특징 표현을 학습한다.